In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class LumpyDataset(Dataset):
    def __init__(self, images, labels) -> None:
        self.X = images
        self.y = labels
        # function for images transformations
        # TODO: try AutoAugment(AutoAugmentPolicy.IMAGENET)
        self.random_transform = transforms.Compose([
            transforms.ToPILImage(),
            # transforms.RandomHorizontalFlip(),
            # transforms.ColorJitter(
            #     brightness=0.3, contrast=0.3),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        num_augment = 3
        augmented_batch = []
        # creating augmented data
        for i in range(num_augment):
            new_item = self.random_transform(self.X[index])
            augmented_batch.append(new_item)
        # labels with one-hot encoding
        label = torch.Tensor([self.y[index]])

        new_labels = [label, label, label]

        return torch.stack(augmented_batch), torch.stack(new_labels)

In [3]:
# loading all dataloaders
with open("./variables/train_loader.pickle", "rb") as f:
    train_loader = pickle.load(f)
with open("./variables/valid_loader.pickle", "rb") as f:
    valid_loader = pickle.load(f)
with open("./variables/test_loader.pickle", "rb") as f:
    test_loader = pickle.load(f)

In [4]:
# loading pretrained ResNet50 model
weights = torchvision.models.ResNet50_Weights.DEFAULT
resnet = torchvision.models.resnet50(weights=weights)

In [5]:
# unfreezing model's parameters
for param in resnet.parameters():
    param.requires_grad = True

n_inputs = resnet.fc.in_features
# modifying last layer of model
resnet.fc = nn.Sequential(nn.Linear(n_inputs, 2048),
                          nn.SELU(),
                          nn.Dropout(p=.4),
                          nn.Linear(2048, 2048),
                          nn.SELU(),
                          nn.Dropout(p=0.4),
                          nn.Linear(2048, 1))

for name, child in resnet.named_children():
    for name2, params in child.named_parameters():
        params.requires_grad = True

In [6]:
# use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [7]:

resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [43]:
# defining loss function and optimizer
lossfunc = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adam(resnet.parameters(), lr=3e-4)

train_losses, test_losses, train_correct, test_correct = [], [], [], []

In [9]:
def save_checkpoint(state, is_best, filename="./best_models/resnet_checkpoint.pth.tar"):
    if is_best:
        torch.save(state, filename)

In [55]:
class EarlyStopping:
    def __init__(self, max_counter, threshold) -> None:
        self.stop_counter = 0
        self.last_best = 0
        self.max_counter = max_counter
        self.threshold = threshold
    
    def stop_learning(self, accuracy):
        if accuracy <= self.last_best or abs(accuracy - self.last_best) <= self.threshold:
            self.stop_counter += 1
            print("MODEL DID NOT IMPROVE FROM", self.last_best)
        else:
            self.stop_counter = 0

        if self.stop_counter == self.max_counter:
            return True
        return False

In [57]:
epochs = 20

earlystopping = EarlyStopping(3, 0.2)

start_time = time.time()

best_prec1 = 2

batch_num, train_b, test_b = None, None, None

for epoch_i in range(epochs):
    train_corr, test_corr = 0, 0
    total_num = 0

    epoch_start = time.time()
    batch_corr = 0
    for batch_num, (X, y) in enumerate(train_loader):
        total_num += 3 * 8
        X, y = X.to(device), y.to(device)

        y_pred = resnet(X.view(-1, 3, 225, 225))
        loss = lossfunc(y_pred, y.view(8*3, -1)) # 16 samples in batch, 3 images in one sample
        # find predicted labels
        predicted = y_pred.view(8*3) > .5

        # calculate the amount of correct predictions in batch
        batch_corr += (predicted.long() == y.view(8*3)).sum()
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_corr = batch_corr.item() / (total_num)
    epoch_end = time.time() - epoch_start
    print(batch_corr)
    print(f"Epoch {epoch_i + 1} | Batch {(batch_num + 1) * 16}\nAccuracy: {train_corr:2.2f} | Loss: {loss.item():2.4f} | Duration: {epoch_end / 60:.2f} minutes")

    train_b = batch_num
    train_losses.append(loss)
    train_correct.append(train_corr)

    X, y = None, None

    with torch.no_grad():
        total_num = 0
        for batch_num, (X, y) in enumerate(valid_loader):
            total_num += 3 * 8
            X, y = X.to(device), y.to(device)

            y_val = resnet(X.view(-1, 3, 225, 225))
            predicted = y_val.view(8*3) > .5

            test_corr += (predicted.long() == y.view(8*3)).sum()
            print((predicted.long() == y.view(8*3)).sum())
    
    loss = lossfunc(y_val, y.view(8*3, 1))

    val_acc = test_corr.item() / (total_num)

    print(f"Validation Accuracy: {val_acc * 100:2.2f} | Validation loss: {loss.item():2.4f}\n")

    is_best = val_acc < best_prec1
    best_prec1 = min(val_acc, best_prec1)
    save_checkpoint({
        "epoch": epoch_i + 1,
        "state_dict": resnet.state_dict(),
        "best_prec1": best_prec1,
    }, is_best)

    test_b = batch_num
    test_losses.append(loss)
    test_correct.append(val_acc)

    if earlystopping.stop_learning(val_acc):
        break

end_time = time.time() - start_time

print(f"\nTraining Duration: {end_time / 60:.2f} minutes")
print(f"GPU memory used: {torch.cuda.memory_allocated()} kb")
print(f"GPU memory cached: {torch.cuda.memory_cached()} kb")

tensor(2162, device='cuda:0')
Epoch 1 | Batch 1488
Accuracy: 0.97 | Loss: 0.0089 | Duration: 5.23 minutes
tensor(20, device='cuda:0')
tensor(21, device='cuda:0')
tensor(18, device='cuda:0')
tensor(24, device='cuda:0')
tensor(19, device='cuda:0')
tensor(18, device='cuda:0')
tensor(21, device='cuda:0')
tensor(19, device='cuda:0')
tensor(24, device='cuda:0')
tensor(18, device='cuda:0')
tensor(18, device='cuda:0')
Validation Accuracy: 83.33 | Validation loss: 0.7814

tensor(2171, device='cuda:0')
Epoch 2 | Batch 1488
Accuracy: 0.97 | Loss: 0.0004 | Duration: 4.94 minutes
tensor(18, device='cuda:0')
tensor(11, device='cuda:0')
tensor(24, device='cuda:0')
tensor(18, device='cuda:0')
tensor(24, device='cuda:0')
tensor(24, device='cuda:0')
tensor(18, device='cuda:0')
tensor(19, device='cuda:0')
tensor(23, device='cuda:0')
tensor(18, device='cuda:0')
tensor(21, device='cuda:0')
Validation Accuracy: 82.58 | Validation loss: 0.2917

tensor(2210, device='cuda:0')
Epoch 3 | Batch 1488
Accuracy: 0.9

c:\Users\husai\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\cuda\memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [58]:
# test model with no gradient updates
with torch.no_grad():
    correct = 0
    test_loss = []
    test_corr = 0
    labels = []
    pred = []
    counter = 0
    # perform test set evaluation batch wise
    for (X, y) in test_loader:
        # set label to use CUDA if available
        X, y = X.to(device), y.to(device)

        # append original labels
        labels.append(y.view(-1, 1))

        y_pred = resnet(X.view(-1, 3, 225, 225))
        loss = lossfunc(y_pred, y.view(8*3, -1)) # 16 samples in batch, 3 images in one sample

        # get argmax of predicted values, which is our label
        predicted = (y_val.view(8*3) > .5).long()
        # append predicted label
        pred.append(y_val)

        # increment correct with correcly predicted labels per batch
        test_corr += (predicted == y.view(8*3)).sum()
        print(test_corr)

        test_loss.append(loss)
        counter += 1
print(f"Test Loss: {test_loss[-1].item():.4f}")
print(f"Test accuracy: {test_corr * 100 / (3*8*counter)}")

tensor(18, device='cuda:0')
tensor(30, device='cuda:0')
tensor(51, device='cuda:0')
tensor(69, device='cuda:0')
tensor(78, device='cuda:0')
tensor(90, device='cuda:0')
tensor(99, device='cuda:0')
tensor(114, device='cuda:0')
tensor(126, device='cuda:0')
tensor(138, device='cuda:0')
tensor(147, device='cuda:0')
Test Loss: 2.7835
Test accuracy: 55.681819915771484
